In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
import pandas as pd
import numpy as np
import json
import fileinput
from tqdm import tqdm
import string
import re
import json



Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [2]:
who_player = pd.DataFrame()
with fileinput.input(files='../' + "whoscored_crawler/player_info_whoscored.jl") as file:
    for line in file:
        conv = json.loads(line)
        who_player = who_player.append(conv, ignore_index=True)
who_player.head()

,player_url,Name,Shirt Number,Age,Height,Nationality,Positions,Strengths,Weaknesses,Style_of_Play,Current Team
0,https://www.whoscored.com/Players/4384/Show/Di...,Didier Drogba,11,43 years old (11-03-1978),189cm,Ivory Coast,Forward,"[Long shots, Direct free-kicks, Finishing, Aer...","[Holding on to the ball, Passing, Discipline]","[Plays the ball off the ground often, Indirect...",NaN
1,https://www.whoscored.com/Players/8040/Show/Ce...,Cesc Fàbregas,4,34 years old (04-05-1987),180cm,Spain,Forward,"[Through balls, Taking set-pieces, Key passes,...","[Finishing, Tackling, Discipline]","[Plays the ball off the ground often, Does not...",Monaco
2,https://www.whoscored.com/Players/4564/Show/Ro...,Robin van Persie,10,38 years old (06-08-1983),183cm,Netherlands,Forward,"[Direct free-kicks, Long shots, Key passes, Dr...","[Tackling, Discipline, Defensive contribution]","[Likes to play short passes, Likes to dribble,...",NaN
3,https://www.whoscored.com/Players/109/Show/Luí...,Luís Boa Morte Pereira,35,44 years old (04-08-1977),178cm,Portugal,Forward,NaN,NaN,NaN,NaN
4,https://www.whoscored.com/Players/48/Show/Fran...,Frank Lampard,8,43 years old (20-06-1978),184cm,England,Midfielder (Centre),[Finishing],"[Tackling, Passing]","[Likes to do layoffs, Does not dive into tackles]",NaN


In [3]:
world_player = pd.DataFrame()
with fileinput.input(files='../' + "worldfootball_crawler/player_info_worldfootball.jl") as file:
    for line in file:
        conv = json.loads(line)
        world_player = world_player.append(conv, ignore_index=True)
world_player.head()

,team_history,player_url,name,Born,Placeofbirth,Nationality,Height,Weight,Position(s),Foot,Current_team_info,Homepage,sizeofshoe
0,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,[24.02.1988],"[Urmston, England]",[England],[183cm],[79kg],[DefensiveMidfielder],[right],NaN,NaN,NaN
1,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,[28.04.1983],"[Ashford, England]",[England],[191cm],[78kg],[CentreBack],[right],NaN,NaN,NaN
2,"{'1': {'period': '07/2020 - 08/2020', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,[17.09.1991],"[Wolverhampton, England]",[England],[191cm],NaN,"[RightWinger, CentreForward]",[right],"{'team_name': 'SJK Seinäjoki', 'country': 'Fin...",NaN,NaN
3,"{'0': {'period': '01/2019 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/p...,Paul Scharner,[11.03.1980],"[Scheibbs, Austria]",[Austria],[191cm],[81kg],"[CentreBack, DefensiveMidfielder]",[right],NaN,[www.paulscharner.at],NaN
4,"{'0': {'period': '05/2013 - 06/2013', 'team': ...",https://www.worldfootball.net/player_summary/d...,David Bentley,[27.08.1984],"[Peterborough, England]",[England],[175cm],[68kg],"[RightMidfielder, RightWinger]",[right],NaN,NaN,NaN


In [4]:
len(who_player)

2155

In [5]:
len(world_player)

2832

In [6]:
                       
len(who_player["player_url"].drop_duplicates())

2155

## ID, Map Generation

In [7]:
len(world_player["player_url"])

2832

In [8]:
len(world_player["player_url"].drop_duplicates())

2832

In [9]:
world_player.groupby("name").count()["player_url"].value_counts()

1    2812
2      10
Name: player_url, dtype: int64

In [10]:
id2player = dict()
player2id = dict()

for idx, player_url in enumerate(list(world_player["player_url"].drop_duplicates())):
    id2player[idx + 10**6] = player_url
    player2id[player_url] = idx + 10**6

for idx, player_url in enumerate(list(who_player["player_url"].drop_duplicates())):
    id2player[idx + 2*(10**6)] = player_url
    player2id[player_url] = idx + 2 * (10**6)

In [11]:
len(who_player["player_url"])

2155

In [12]:
len(who_player["player_url"].drop_duplicates())

2155

In [13]:
len(id2player)

4987

## Blocking

In [14]:
def f(x):
    if type(x) == float:
        return 0
    elif len(x[0])<15:
        tmp = x[0].split(".")
        return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))
    else:
        tmp = x[0].split(".")
        return int(str(tmp[2])[:4] + str(tmp[1]) + str(tmp[0]))
world_player["birth_block"] = world_player["Born"].apply(f)

In [15]:
who_player["DOB"] = who_player["Age"].apply(lambda x : x.split(" years old (")[1][:-1])

who_player["Age"] = who_player["Age"].apply(lambda x : x.split(" years old (")[0])

In [16]:
def f(x):
    tmp = x.split("-")
    return int(str(tmp[2]) + str(tmp[1]) + str(tmp[0]))
who_player["birth_block"] = who_player["DOB"].apply(f)

In [17]:
who_player['player_id'] = who_player["player_url"].apply(lambda x : player2id[x])
world_player['player_id'] = world_player["player_url"].apply(lambda x : player2id[x])

In [18]:
who_birth = who_player[["player_id", "birth_block"]]
world_birth = world_player[["player_id", "birth_block"]]
birth_df = who_birth.append(world_birth)

In [19]:
# birth_df.groupby("birth_block").count().sort_values(by = "player_id", ascending = False)
birth_df

,player_id,birth_block
0,2000000,19780311
1,2000001,19870504
2,2000002,19830806
3,2000003,19770804
4,2000004,19780620
...,...,...
2827,1002827,20010203
2828,1002828,19911123
2829,1002829,19910411
2830,1002830,19840922


In [20]:
world_name = world_player[["player_id", "name"]]
who_name = who_player[["player_id", "Name"]].rename({"Name" : "name"}, axis =1 )
name_df = world_name.append(who_name)

# Matching

In [21]:
tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
match_dict =dict()
match_dict2 = dict()
id2name = name_df.set_index('player_id').to_dict()["name"]
Lev = NormalizedLevenshtein().similarity

for idx, row in tmp.iterrows():
    if idx != 0 and len(row.player_id)>1:
        name_list_1 = []
        name_list_2 = []
        for idx in row.player_id:
            if str(idx)[0] == "1":
                name_list_1.append((idx,id2name[idx].lower()))
            if str(idx)[0] == "2":
                name_list_2.append((idx,id2name[idx].lower()))
        if len(name_list_1) == 0 or len(name_list_2) == 0:
            continue
        else:
            for (idx1, name1) in name_list_1:
                for (idx2, name2) in name_list_2:
                    if Lev(name1, name2) > 0.4:
#                         if Lev(name1, name2) < 0.5:
#                             print(name1 +"\t" + name2 + "\t" + str(Lev(name1, name2)))
                        try:
                            a = match_dict[idx1]
                            if Lev(name1, name2) > Lev(name1, a[0][1]):
                                match_dict[idx1] = [(idx2, name2)]
                                match_dict2[idx2] = [(idx1, name1)]
                        except:
                            match_dict[idx1] = match_dict.get(idx1, []) + [(idx2, name2)]
                            match_dict2[idx2] = match_dict2.get(idx2, []) + [(idx1, name1)]
        
            
                        
#                         match_dict[idx1] = match_dict.get(idx1, []) + [(idx2, name2)]
#                         match_dict2[idx2] = match_dict2.get(idx2, []) + [(idx1, name1)]
        
        
                        

In [22]:
id_list1 = [i for i in list(id2player.keys()) if str(i)[0] == "1" and i in match_dict.keys()]

In [23]:
id_list2 = [i for i in list(id2player.keys()) if str(i)[0] == "2" and i in match_dict2.keys()]

In [24]:
from collections import Counter

In [25]:
tmp = []
for i in list(match_dict.values()):
    tmp+=i

In [26]:
tmp = []
for i in list(match_dict2.values()):
    tmp+=i

In [27]:
len(id_list1)

1945

In [28]:
len(id_list2)

1941

In [29]:
len(match_dict)

1945

In [30]:
[i for i in match_dict2.items() if len(i[1])!=1]

[(2000222, [(1000344, 'tyrone mears'), (1000504, 'jermaine jenas')]),
 (2000207, [(1000473, 'martin olsson'), (1000994, 'marcus olsson')]),
 (2000018, [(1000036, 'ciaran clark'), (1000569, 'kieran gibbs')]),
 (2001079, [(1001342, 'muniesa'), (1002597, 'mark gillespie')]),
 (2000872, [(1000971, 'will keane'), (1000972, 'michael keane')]),
 (2001668, [(1001100, 'charles banya'), (1002217, 'charlie taylor')]),
 (2001187, [(1001343, 'josh murphy'), (1002212, 'jacob murphy')]),
 (2001329, [(1001527, 'josé pozo'), (1001741, 'josh doherty')]),
 (2001835, [(1002401, 'steven sessegnon'), (1002404, 'ryan sessegnon')])]

In [31]:
k = [i for i in match_dict.items() if len(i[1])!=1]
k

[]

In [32]:
def f(x):
    try:
        return match_dict[x][0][0]
    except:
        return "No match"
world_player["match_player_id"] = world_player["player_id"].apply(f)

In [33]:
# outer join in python pandas
outer_join_df=pd.merge(world_player, who_player, left_on = "match_player_id", right_on = "player_id", how='outer')
outer_join_df


,team_history,player_url_x,name,Born,Placeofbirth,Nationality_x,Height_x,Weight,Position(s),Foot,...,Height_y,Nationality_y,Positions,Strengths,Weaknesses,Style_of_Play,Current Team,DOB,birth_block_y,player_id_y
0,"{'0': {'period': '07/2011 - 06/2012', 'team': ...",https://www.worldfootball.net/player_summary/m...,Michael Johnson,[24.02.1988],"[Urmston, England]",[England],[183cm],[79kg],[DefensiveMidfielder],[right],...,183cm,England,Midfielder,NaN,NaN,NaN,NaN,24-02-1988,19880224.0,2000534.0
1,"{'0': {'period': '10/2017 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/r...,Roger Johnson,[28.04.1983],"[Ashford, England]",[England],[191cm],[78kg],[CentreBack],[right],...,191cm,England,Defender (Centre),[Blocking the ball],[Passing],"[Indirect set-piece threat, Likes to play long...",NaN,28-04-1983,19830428.0,2000086.0
2,"{'1': {'period': '07/2020 - 08/2020', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Jervis,[17.09.1991],"[Wolverhampton, England]",[England],[191cm],NaN,"[RightWinger, CentreForward]",[right],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'1': {'period': '07/2016 - 06/2019', 'team': ...",https://www.worldfootball.net/player_summary/j...,Jake Bidwell,[21.03.1993],"[Southport, England]",[England],[183cm],[80kg],"[LeftBack, LeftMidfielder]",[left],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'0': {'period': '07/2013 - 06/2016', 'team': ...",https://www.worldfootball.net/player_summary/b...,Bosingwa,[24.08.1982],"[Mbandaka, CongoDR]","[Portugal, CongoDR]",[183cm],[75kg],"[RightBack, RightMidfielder]",[right],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190cm,Brazil,Defender (Centre),[Passing],[],"[Likes to play short passes, Commits fouls often]",Arsenal,19-12-1997,19971219.0,2002066.0
3042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,178cm,Brazil,Midfielder (Left),"[Holding on to the ball, Tackling, Concentration]",[],[Commits fouls often],Wolverhampton Wanderers,19-02-1989,19890219.0,2002098.0
3043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190cm,NaN,Forward,"[Long shots, Defensive contribution]","[Finishing, Aerial Duels, Holding on to the ball]",[Likes to dribble],Burton,13-08-2003,20030813.0,2002099.0
3044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,175cm,Croatia,"Midfielder (Centre, Left)",[Tackling],[],"[Gets fouled often, Likes to do layoffs, Likes...",Hajduk Split,29-08-1995,19950829.0,2002105.0


In [34]:
outer_join_df.to_csv("../data_" + "player_join.csv")

# PSL DATA Generation


In [35]:
data_dir = 'PSL_data/'

In [36]:
# count = []
# for i in birth_df.groupby("birth_block")["player_id"].apply(list):
#     list1 = []
#     list2 = []
#     if len(i)>1:
#         for ele in i:
#             if str(ele)[0] == '1':
#                 list1.append(ele)
#             else:
#                 list2.append(ele)

#             count.append(len(list1) * len(list2))

In [37]:
# max(count)

In [38]:
# count1 = 0
# count2 = 0
# for i in list(id2player.keys()):
#     if str(i)[0] == '1':
#         count1 += 1
#     else:
#         count2 += 1

In [39]:
## Birth Block
with open(data_dir + "birthBlock_obs.txt", "w") as f:
    for idx, row in birth_df.iterrows():
        f.write(str(row.player_id) + "\t" + str(row.birth_block) + "\n")

In [40]:
# count1*count2

In [41]:
## Name sim
Lev = NormalizedLevenshtein().similarity
with open(data_dir + "simName_obs.txt", "w") as f:
    tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
    for id_list in tmp["player_id"]:
        for idx1 in id_list:
            for idx2 in id_list:
                name1 = str(list(name_df[name_df["player_id"] == int(idx1)]["name"])[0])
                name2 = str(list(name_df[name_df["player_id"] == int(idx2)]["name"])[0])
                f.write(str(idx1) + "\t" + str(idx2) + "\t" + str(Lev(name1.lower(), name2.lower() )) + "\n")

with open(data_dir + "simName_obs.txt", "r") as f:
    simName = f.read()

with open(data_dir + "simName_obs.txt", "w") as f:
    for i in set(simName.split("\n")):
        if i != "":
            f.write(i + "\n")

In [42]:
# 1-(count/(count1*count2))

In [43]:
## target
with open(data_dir + "same_player_target.txt", "w") as f:
    tmp = pd.DataFrame(birth_df.groupby("birth_block")["player_id"].apply(list))
    for id_list in tmp["player_id"]:
        for idx1 in id_list:
            for idx2 in id_list:
                f.write(str(int(idx1)) + "\t" + str(int(idx2)) + "\n")
                
with open(data_dir + "same_player_target.txt", "r") as f:
    data = f.read()

with open(data_dir + "same_player_target.txt", "w") as f:
    for i in set(data.split("\n")):
        if i != "":
            f.write(i + "\n")



In [44]:
# tmp = world_player[["name","player_url"]]
world_player["player_id"] = world_player["player_url"].apply(lambda x : player2id[x])
who_player["player_id"] = who_player["player_url"].apply(lambda x : player2id[x])


In [45]:
## playerName_obs.txt
with open("PSL_data/playerName_obs.txt", "w") as f:
    for idx, player in id2player.items():
        f.write(str(idx) + "\t" + player + "\n")